<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Transformación

En este laboratorio, tu objetivo será limpiar y preparar los datos extraídos previamente de diferentes fuentes para su posterior análisis. Trabajarás con datos provenientes de la API de Red Eléctrica Española (REE) y del Instituto Nacional de Estadística (INE). Estos datos incluyen información sobre demanda y generación eléctrica a nivel provincial, así como datos demográficos y económicos. 






In [178]:
# Para gestionar ficheros
# -----------------------------------------------------------------------
import os
import glob

# Para trabajar con access
# -----------------------------------------------------------------------
import subprocess
import json
# Para trabajar con regex
# -----------------------------------------------------------------------
import re
# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os
# Para trabajar con los DataFrames
# -----------------------------------------------------------------------
import pandas as pd
# Para poner barras progreso
# -----------------------------------------------------------------------
from tqdm import tqdm
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")


- Cargar los Datos Extraídos:

  - **Demanda Eléctrica:** Carga los datos de demanda eléctrica extraídos de la API de REE.



In [179]:
ruta_og = "datos/DatosDescargados/RedElectrica/EvolucionDemanda/"
years = os.listdir(ruta_og)
df_demanda = pd.DataFrame()
for year in years:
    ruta_years = os.path.join(ruta_og,year) # Entrar a cada carpeta
    archivos = os.listdir(ruta_years)
    comunidads = []
    yeardatosl = []
    for archivo in archivos:
        comunidad = archivo.split("_")[1]
        comunidads.append(comunidad)
        yeardatos = archivo.split("_")[2].strip(".json")
        yeardatosl.append(yeardatos)
        communi = []
        fechas = []
        porcentaje = []
        demanda = []
        with open(f"{ruta_years}/{archivo}") as f:
            data = json.load(f)

        for valor in data["included"][0]["attributes"]["values"]:
            communi.append(comunidad)
            demanda.append(valor["value"])
            porcentaje.append(valor["percentage"])
            fechas.append(valor["datetime"])
    
        df_temp = pd.DataFrame({
            "comunidad_autónoma" : communi,
            "demanda (MW)" : demanda,
            "porcentaje (%)" : porcentaje,
            "fecha" : fechas
        })
        df_demanda = pd.concat([df_demanda,df_temp])
            
df_demanda.sample(5)
        

,comunidad_autónoma,demanda (MW),porcentaje (%),fecha
4,Principado de Asturias,776093.729,1,2021-05-01T00:00:00.000+02:00
5,Canarias,714689.013,1,2019-06-01T00:00:00.000+02:00
6,Región de Murcia,934973.704,1,2019-07-01T00:00:00.000+02:00
10,Comunidad Foral de Navarra,437375.677,1,2021-11-01T00:00:00.000+01:00
3,Región de Murcia,706783.625,1,2021-04-01T00:00:00.000+02:00


In [180]:
df_demanda.shape

(684, 4)

  - **Generación Eléctrica:** Carga los datos de generación eléctrica diferenciados por tipo de energía (eólica, solar, hidroeléctrica, etc.) a nivel provincial.


In [181]:
ruta_og = "datos/DatosDescargados/RedElectrica/GeneracionElectrica/"
years = os.listdir(ruta_og)
df_generacion = pd.DataFrame()
for year in years:
    ruta_years = os.path.join(ruta_og,year) # Entrar a cada carpeta
    archivos = os.listdir(ruta_years)
    comunidads = []
    yeardatosl = []
    for archivo in archivos:
        comunidad = archivo.split("_")[2]
        comunidads.append(comunidad)
        yeardatos = archivo.split("_")[3].strip(".json")
        yeardatosl.append(yeardatos)
        communi = []
        fechas = []
        porcentaje = []
        generacion = []
        categoria = []
        with open(f"{ruta_years}/{archivo}") as f:
            data = json.load(f)
        
        for datos in data["included"]:
            for valor in datos["attributes"]["values"]:
                communi.append(comunidad)
                categoria.append(datos["type"])
                generacion.append(valor["value"])
                porcentaje.append(valor["percentage"])
                fechas.append(valor["datetime"])
            
            df_temp = pd.DataFrame({
            "comunidad_autónoma" : communi,
            "categoria" : categoria,
            "generacion (MW)" : generacion,
            "porcentaje (%)" : porcentaje,
            "fecha" : fechas
            })
            df_generacion = pd.concat([df_generacion,df_temp])

df_generacion.head()
            


,comunidad_autónoma,categoria,generacion (MW),porcentaje (%),fecha
0,Andalucía,Hidráulica,25288.605,0.029517,2019-01-01T00:00:00.000+01:00
1,Andalucía,Hidráulica,33933.350,0.030779,2019-02-01T00:00:00.000+01:00
2,Andalucía,Hidráulica,35913.000,0.029645,2019-03-01T00:00:00.000+01:00
3,Andalucía,Hidráulica,44050.848,0.041340,2019-04-01T00:00:00.000+02:00
4,Andalucía,Hidráulica,88535.452,0.068497,2019-05-01T00:00:00.000+02:00



  - **Datos Demográficos:** Carga los datos demográficos por provincia extraídos de la web del INE.


In [182]:
df_Demograficos = pd.read_csv("datos/DatosDescargados/DatosInePorProvincias2019-2021.csv",encoding='latin-1', sep = ';')
df_Demograficos.head()

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
0,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2021,47.385.107
1,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2020,47.450.795
2,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2019,47.026.208
3,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Hombres,2021,23.222.953
4,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Hombres,2020,23.255.590



  - **Datos Económicos:** Carga los datos del PIB por provincia obtenidos del INE.

In [183]:
df_pib = pd.read_csv("datos/DatosDescargados/DatosPIBPorProvincias2019-2021.csv",encoding='latin-1', sep = ';')
df_pib.head()

,Provincias,Ramas de actividad,Magnitud,periodo,Total
0,02 Albacete,TOTAL PERSONAS,Empleo total (miles de personas),2021(P),"150,2"
1,02 Albacete,TOTAL PERSONAS,Empleo total (miles de personas),2020,"147,1"
2,02 Albacete,TOTAL PERSONAS,Empleo total (miles de personas),2019,"151,9"
3,02 Albacete,TOTAL PERSONAS,Empleo total (miles de personas),2018,"148,3"
4,02 Albacete,TOTAL PERSONAS,Empleo asalariado (miles de personas),2021(P),"124,7"


-Limpieza de Datos:


In [184]:
ruta_final = "datos/DatosFinales/RedElectrica"
os.makedirs(ruta_final,exist_ok=True)


- Datos de la API de REE:

  - **Demanda Eléctrica:**

    - **Conversión de Timestamps:** Asegúrate de que las fechas estén correctamente formateadas en `datetime`. Si es necesario, convierte los datos a un formato uniforme (por ejemplo, `YYYY-MM` para datos mensuales).

    - **Tratamiento de Valores Nulos:** Identifica y maneja los valores nulos en caso de que los haya. Puedes optar por eliminar filas con valores faltantes.

    - **Estandarización de Nombres de Provincias:** Verifica que los nombres de las provincias estén estandarizados y coincidan en todos los conjuntos de datos. Si hay inconsistencias, corrígelas.

 



In [185]:
df_demanda.head()

,comunidad_autónoma,demanda (MW),porcentaje (%),fecha
0,Andalucía,3668893.328,1,2019-01-01T00:00:00.000+01:00
1,Andalucía,3152278.091,1,2019-02-01T00:00:00.000+01:00
2,Andalucía,3216356.360,1,2019-03-01T00:00:00.000+01:00
3,Andalucía,3029639.973,1,2019-04-01T00:00:00.000+02:00
4,Andalucía,3154123.467,1,2019-05-01T00:00:00.000+02:00


In [186]:
df_demanda["fecha"]=df_demanda["fecha"].str.split("T",expand=True)[0]
df_demanda["fecha"] = pd.to_datetime(df_demanda["fecha"])

In [187]:
df_demanda.info()

<class 'pandas.core.frame.DataFrame'>
Index: 684 entries, 0 to 11
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   comunidad_autónoma  684 non-null    object        
 1   demanda (MW)        684 non-null    float64       
 2   porcentaje (%)      684 non-null    int64         
 3   fecha               684 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 26.7+ KB


In [188]:
df_demanda.head()

,comunidad_autónoma,demanda (MW),porcentaje (%),fecha
0,Andalucía,3668893.328,1,2019-01-01
1,Andalucía,3152278.091,1,2019-02-01
2,Andalucía,3216356.360,1,2019-03-01
3,Andalucía,3029639.973,1,2019-04-01
4,Andalucía,3154123.467,1,2019-05-01


# Dejamos mes y año en vez de fecha

In [342]:
df_demanda["mes"] = df_demanda["fecha"].dt.month
df_demanda["año"] = df_demanda["fecha"].dt.year
df_demanda = df_demanda.drop(columns="fecha")

In [343]:
df_demanda.head()

,comunidad_autónoma,demanda (MW),porcentaje (%),mes,año
0,Andalucía,3668893.3280,1,1,2019
1,Andalucía,3152278.0910,1,2,2019
2,Andalucía,3216356.3600,1,3,2019
3,Andalucía,3029639.9730,1,4,2019
4,Andalucía,3154123.4670,1,5,2019


In [344]:
df_demanda.to_csv("datos/DatosFinales/RedElectrica/DemandaElectrica.csv")

 - **Generación Eléctrica:**

    - **Desagregación de Tecnologías:** Asegúrate de que los datos estén correctamente desglosados por tipo de energía. Revisa que los campos correspondientes a energía eólica, solar, hidroeléctrica, etc., estén bien identificados y sin errores.

    - **Normalización de Unidades:** Verifica que las unidades de energía estén estandarizadas (por ejemplo, MWh). Realiza las conversiones necesarias si se encuentran en otras unidades.

    - **Identificación de Outliers:** Revisa los valores extremos o atípicos en la generación de energía y decide si deben ser tratados o eliminados.

In [346]:
df_generacion.head()

,comunidad_autónoma,categoria,generacion (MW),porcentaje (%),fecha
0,Andalucía,Hidráulica,25288.6050,0.0295,2019-01-01
1,Andalucía,Hidráulica,33933.3500,0.0308,2019-02-01
2,Andalucía,Hidráulica,35913.0000,0.0296,2019-03-01
3,Andalucía,Hidráulica,44050.8480,0.0413,2019-04-01
4,Andalucía,Hidráulica,88535.4520,0.0685,2019-05-01


In [347]:
df_generacion["fecha"]=df_generacion["fecha"].str.split("T",expand=True)[0]
df_generacion["fecha"] = pd.to_datetime(df_generacion["fecha"])

In [348]:
df_generacion.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11786 entries, 0 to 71
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   comunidad_autónoma  11786 non-null  object        
 1   categoria           11786 non-null  object        
 2   generacion (MW)     11786 non-null  float64       
 3   porcentaje (%)      11786 non-null  float64       
 4   fecha               11786 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 552.5+ KB


In [350]:
df_generacion.head()

,comunidad_autónoma,categoria,generacion (MW),porcentaje (%),fecha
0,Andalucía,Hidráulica,25288.6050,0.0295,2019-01-01
1,Andalucía,Hidráulica,33933.3500,0.0308,2019-02-01
2,Andalucía,Hidráulica,35913.0000,0.0296,2019-03-01
3,Andalucía,Hidráulica,44050.8480,0.0413,2019-04-01
4,Andalucía,Hidráulica,88535.4520,0.0685,2019-05-01


In [351]:
df_generacion.drop_duplicates(inplace=True)
df_generacion.reset_index(drop=True,inplace=True)

# Separo la fecha por mes y año

In [352]:
df_generacion["mes"] = df_generacion["fecha"].dt.month
df_generacion["año"] = df_generacion["fecha"].dt.year
df_generacion = df_generacion.drop(columns="fecha")

# Quito generacion renovable
Representa el total y lo prefiero calcular segun los datos que proveen

In [353]:
condicion = df_generacion["categoria"] != "Generación renovable"
df_generacion = df_generacion[condicion]

# Calculo el porcentaje de nuevo

Ademas lo voy a hacer del 100% para tener valores mas entendibles   

In [354]:
pd.options.display.float_format = '{:.4f}'.format
cond_2019 = df_generacion["año"] == 2019
cond_2020 = df_generacion["año"] == 2020
cond_2021 = df_generacion["año"] == 2021
df_gen2019 = df_generacion[cond_2019]
df_gen2020 = df_generacion[cond_2020]
df_gen2021 = df_generacion[cond_2021]

In [355]:

total_mes_comunidad2019 = df_gen2019.groupby(["mes", "comunidad_autónoma"])["generacion (MW)"].sum().to_dict()
total_mes_comunidad2020 = df_gen2020.groupby(["mes", "comunidad_autónoma"])["generacion (MW)"].sum().to_dict()
total_mes_comunidad2021 = df_gen2021.groupby(["mes", "comunidad_autónoma"])["generacion (MW)"].sum().to_dict()

df_gen2019["porcentaje (%)"] = df_gen2019.apply(
    lambda row: row["generacion (MW)"] / total_mes_comunidad2019[(row["mes"], row["comunidad_autónoma"])] * 100, axis=1
)
df_gen2020["porcentaje (%)"] = df_gen2020.apply(
    lambda row: row["generacion (MW)"] / total_mes_comunidad2020[(row["mes"], row["comunidad_autónoma"])] * 100, axis=1
)
df_gen2021["porcentaje (%)"] = df_gen2021.apply(
    lambda row: row["generacion (MW)"] / total_mes_comunidad2021[(row["mes"], row["comunidad_autónoma"])] * 100, axis=1
)

df_generacion = pd.concat([df_gen2019,df_gen2020,df_gen2021]).reset_index(drop=True)
df_generacion

,comunidad_autónoma,categoria,generacion (MW),porcentaje (%),mes,año
0,Andalucía,Hidráulica,25288.6050,2.9517,1,2019
1,Andalucía,Hidráulica,33933.3500,3.0779,2,2019
2,Andalucía,Hidráulica,35913.0000,2.9645,3,2019
3,Andalucía,Hidráulica,44050.8480,4.1340,4,2019
4,Andalucía,Hidráulica,88535.4520,6.8497,5,2019
...,...,...,...,...,...,...
2904,Región de Murcia,Otras renovables,3862.0630,1.6348,8,2021
2905,Región de Murcia,Otras renovables,3315.8530,1.5251,9,2021
2906,Región de Murcia,Otras renovables,3266.0950,1.7445,10,2021
2907,Región de Murcia,Otras renovables,2672.0510,1.3830,11,2021


In [356]:
df_generacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2909 entries, 0 to 2908
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   comunidad_autónoma  2909 non-null   object 
 1   categoria           2909 non-null   object 
 2   generacion (MW)     2909 non-null   float64
 3   porcentaje (%)      2909 non-null   float64
 4   mes                 2909 non-null   int32  
 5   año                 2909 non-null   int32  
dtypes: float64(2), int32(2), object(2)
memory usage: 113.8+ KB


In [357]:
df_generacion.to_csv("datos/DatosFinales/RedElectrica/GeneracionElectrica.csv")

- Datos del INE:

  - **Datos Demográficos:**

    - **Consistencia en la Codificación de Provincias:** Asegúrate de que los nombres de las provincias en los datos demográficos coincidan con los nombres utilizados en los datos eléctricos.

    - **Revisión de Categorías:** Verifica que las categorías de edad, sexo, y nacionalidad estén correctamente etiquetadas y sean consistentes en todo el dataset.

    - **Manejo de Valores Faltantes:** Revisa la presencia de valores faltantes y decide cómo tratarlos (relleno, eliminación o sustitución).


In [358]:
df_demograficos = pd.read_csv("datos/DatosDescargados/DatosInePorProvincias2019-2021.csv",encoding='latin-1', sep = ';')
df_Demograficos.head()

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
0,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2021,47.385.107
1,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2020,47.450.795
2,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2019,47.026.208
3,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Hombres,2021,23.222.953
4,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Hombres,2020,23.255.590


In [361]:
cond1 = df_Demograficos["Provincias"] != "TOTAL ESPAÑA"
cond2 = df_Demograficos["Edad (3 grupos de edad)"] != "TOTAL EDADES"
df_Demograficos = df_Demograficos[cond1 & cond2]
df_Demograficos

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
144,02 Albacete,TOTAL EDADES,TOTAL,Ambos sexos,2021,386.464
145,02 Albacete,TOTAL EDADES,TOTAL,Ambos sexos,2020,388.270
146,02 Albacete,TOTAL EDADES,TOTAL,Ambos sexos,2019,388.167
147,02 Albacete,TOTAL EDADES,TOTAL,Hombres,2021,193.205
148,02 Albacete,TOTAL EDADES,TOTAL,Hombres,2020,194.081
...,...,...,...,...,...,...
7627,52 Melilla,65 y más,% Extranjeros,Hombres,2020,"10,86"
7628,52 Melilla,65 y más,% Extranjeros,Hombres,2019,"10,73"
7629,52 Melilla,65 y más,% Extranjeros,Mujeres,2021,"9,55"
7630,52 Melilla,65 y más,% Extranjeros,Mujeres,2020,"9,55"



  - **Datos Económicos:**

    - **Normalización del PIB:** Si los datos del PIB están en diferentes unidades o escalas, asegúrate de normalizarlos para que sean comparables entre provincias.

    - **Agrupación Temporal:** Si los datos económicos están disponibles en diferentes periodos temporales, agrúpalos y normalízalos para que coincidan con los datos eléctricos en términos de granularidad temporal (mensual o anual).

NOTA: Ten en cuenta que los datos los vamos a tener que insertar en una base de datos mañana, por lo que toda esta limpieza os recomendamos que la penséis para poder crear e insertar los datos mañana. 

In [204]:
df_economicos = pd.read_csv("datos/DatosDescargados/DatosPIBPorProvincias2019-2021.csv",encoding='latin-1', sep = ';')
df_economicos.head()

,Provincias,Ramas de actividad,Magnitud,periodo,Total
0,02 Albacete,TOTAL PERSONAS,Empleo total (miles de personas),2021(P),"150,2"
1,02 Albacete,TOTAL PERSONAS,Empleo total (miles de personas),2020,"147,1"
2,02 Albacete,TOTAL PERSONAS,Empleo total (miles de personas),2019,"151,9"
3,02 Albacete,TOTAL PERSONAS,Empleo total (miles de personas),2018,"148,3"
4,02 Albacete,TOTAL PERSONAS,Empleo asalariado (miles de personas),2021(P),"124,7"
